In [80]:
import requests
import xml.etree.ElementTree as ET
import os
import re
import pickle
from dotenv import load_dotenv
import getpass
from datetime import date
import pandas as pd

In [81]:
# Where to store session:
SESSION_FILE = ".balanz_session.pkl"

def save_session(access_token, cookie, id_cuenta):
    with open(SESSION_FILE, "wb") as f:
        pickle.dump((access_token, cookie, id_cuenta), f)

def load_session():
    if not os.path.exists(SESSION_FILE):
        return None, None, None
    try:
        with open(SESSION_FILE, "rb") as f:
            return pickle.load(f)
    except:
        return None, None, None

def login(username, password):
    session = requests.Session()
    # ---------- 1. Get NONCE ----------
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Origin": "https://clientes.balanz.com",
        "Referer": "https://clientes.balanz.com/auth/login?avoidAuthRedirect=true",
        "Content-Type": "application/json"
    }
    resp = session.post(
        "https://clientes.balanz.com/api/v1/auth/init?avoidAuthRedirect=true",
        json={"user": username}, headers=headers
    )
    print("DEBUG: Nonce status:", resp.status_code)
    print("DEBUG: Nonce headers:", resp.headers)
    print("DEBUG: Nonce response:", resp.text)

    # Parse the nonce value from the XML
    root = ET.fromstring(resp.text)
    nonce = root.findtext("nonce")
    print("DEBUG: Extracted nonce:", nonce)
    if not nonce:
        raise Exception("Could not get nonce")

    # Extract cookiesession1 from SET-COOKIE NOW (not after login)
    raw_cookie = resp.headers.get("Set-Cookie", "")
    m = re.search(r'cookiesession1=([^;]+)', raw_cookie)
    cookiesession1 = m.group(1) if m else None
    print("DEBUG: cookiesession1 after INIT:", cookiesession1)
    if not cookiesession1:
        raise Exception("No cookiesession1 set after init! Something is wrong.")

    # ---------- 2. LOGIN ----------
    login_xml = f"""<jsonObject><user>{username}</user><pass>{password}</pass><nonce>{nonce}</nonce></jsonObject>"""
    headers["Content-Type"] = "application/xml"
    resp2 = session.post(
        "https://clientes.balanz.com/api/v1/auth/login?avoidAuthRedirect=true",
        data=login_xml, headers=headers
    )
    print("\nDEBUG: LOGIN status:", resp2.status_code)
    print("DEBUG: LOGIN headers:", resp2.headers)
    print("DEBUG: LOGIN response (raw):", resp2.text)

    if "Sesiones activas" in resp2.text:
        raise Exception("Too many active sessions. Wait and try again.")
    if "idError" in resp2.text:
        # Optional: extract and print error description
        try:
            root2 = ET.fromstring(resp2.text)
            mensaje = root2.findtext("Descripcion")
            print(f"DEBUG: Login error description: {mensaje}")
        except Exception:
            print("DEBUG: Failed to parse login error XML")
        raise Exception("Login failed.")

    try:
        root2 = ET.fromstring(resp2.text)
    except Exception as e:
        print("DEBUG: Could not parse login XML! Exception:", e)
        print("DEBUG: LOGIN RESPONSE:", resp2.text)
        raise

    access_token = root2.findtext("AccessToken")
    id_cuenta = root2.findtext("idPersona")  # Or adjust if API returns a different field
    print(f"DEBUG: Extracted AccessToken: {access_token}")
    print(f"DEBUG: Extracted id_cuenta/idPersona: {id_cuenta}")

    # Do NOT try to parse Set-Cookie from LOGIN (resp2), only use value from INIT (resp)
    print(f"DEBUG: Login complete. Using AccessToken={access_token}, id_cuenta={id_cuenta}, session cookie={cookiesession1}")

    if not (access_token and cookiesession1 and id_cuenta):
        print("DEBUG: One or more required values missing!")
        raise Exception("Login failed, no access/cookie/id")

    # Optionally return session *object* if you want to use it for requests with cookies attached
    return access_token, cookiesession1, id_cuenta  # Optionally: , session

def session_test_call(access_token, cookie, id_cuenta):
    # Try a lightweight authenticated call; here we use the "cotizacioninstrumento" endpoint as an example.
    url = "https://clientes.balanz.com/api/v1/cotizacioninstrumento"
    headers = {
        "Authorization": access_token,
        "Cookie": f"cookiesession1={cookie}",
        "Accept": "application/json",
        "User-Agent": "Mozilla/5.0"
    }
    params = {
        "plazo": "1",
        "idCuenta": id_cuenta,
        "ticker": "S2S5C"  # Put a valid, but simple/cheap ticker, e.g. known to always work
    }
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=6)
        # If forbidden/expired
        if resp.status_code == 403 and "Sesion Expirada" in resp.text:
            return False
        if resp.status_code == 401:
            return False
        # If error codes in json
        try:
            data = resp.json()
            if "Sesion Expirada" in data.get("Descripcion",""):
                return False
        except Exception:
            pass
        # Otherwise session OK
        return True
    except:
        return False

def get_balanz_session(username, password):
    access_token, cookie, id_cuenta = load_session()
    print(f"DEBUG: Loaded session: AccessToken={access_token}, cookie={cookie}, id_cuenta={id_cuenta}") # DEBUG
    if access_token and cookie and id_cuenta:
        ok = session_test_call(access_token, cookie, id_cuenta)
        print(f"DEBUG: session_test_call returned: {ok}") # DEBUG
        if ok:
            print("Using previous session.")
            return access_token, cookie, id_cuenta
        else:
            print("Session expired or invalid, logging in again...")
    else:
        print("No stored session, logging in...")

    access_token, cookie, id_cuenta = login(username, password)
    print("New session obtained.")
    save_session(access_token, cookie, id_cuenta)
    return access_token, cookie, id_cuenta

In [82]:
# Load .env variables into environment
load_dotenv(override=True)

# Read credentials
username = os.getenv("BALANZ_USER")
password = os.getenv("BALANZ_PASSWORD")

if not username:
    username = input("Please enter Balanz username: ")

if not password:
    password = getpass.getpass("Please enter Balanz password: ")

access_token, cookie, id_cuenta = get_balanz_session(username, password)
# You are now logged in and ready to use these credentials!

DEBUG: Loaded session: AccessToken=6E08672F-266B-42F9-81DE-314216D2A358, cookie=678ADAB85CA0A151E0C735830928F175, id_cuenta=1205083
DEBUG: session_test_call returned: True
Using previous session.


In [83]:
def get_ticker_data(ticker):
    url = "https://clientes.balanz.com/api/v1/historico/relacion"
    params = {
        "ticker": ticker,
        "plazo": "1"
    }
    headers = {
        "Accept": "application/json",
        "Authorization": access_token,
        "Cookie": f"cookiesession1={cookie}",
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers, params=params)

    try:
        # Try to decode JSON if possible
        return response.json()
    except Exception:
        print("Raw response:", response.text[:500])  # In case it's not JSON

In [84]:
def get_cupon(ticker):
    url = "https://clientes.balanz.com/api/v1/cotizacioninstrumento"
    params = {
        "ticker": ticker,
        "plazo": "1",
        "fullNormalize": "false"
    }
    headers = {
        "Accept": "application/json",
        "Authorization": access_token,
        "Cookie": f"cookiesession1={cookie}",
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers, params=params)

    print("Status code:", response.status_code)
    try:
        return response.json()
    except Exception:
        print("Raw response:\n", response.text[:500])

In [85]:
url = "https://clientes.balanz.com/api/v1/cotizaciones/letras"
params = {
    "token": "0",
    "tokenindice": "0"
}
headers = {
    "Accept": "application/json",
    "Authorization": access_token,
    "Cookie": f"cookiesession1={cookie}",
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers, params=params)

print("Status code:", response.status_code)
try:
    print("JSON response:", response.json())
except Exception as e:
    print("Failed to decode JSON. Raw response:\n", response.text[:500])

Status code: 200
JSON response: {'deferredLevel': 0, 'plazoDefault': 1, 'indice': [], 'cotizaciones': [{'id': 'S2S5C-0001-C-CT-EXT', 'deferredLevel': 0, 'SecurityID': 'S2S5C-0001-C-CT-EXT', 'ticker': 'S2S5C', 'descripcion': 'LETRAS DEL TESORO CAP $ V30/09/25', 'mo': 'Dolar Cable', 'idMo': 4, 'idPlazo': 0, 'plazo': 'CI', 'panel': 15, 'u': 0.00107, 'v': 0, 'ant': 0.00107, 'pc': -1, 'cc': -1, 'pv': 0.00107, 'cv': 95150000, 'max': 0.00107, 'min': 0.00107, 'ap': 0.00107, 'idWatchlist': 0, 't': 1758284212674, 'Tipo': 'Letras', 'IdTipo': 15, 'MarketId': 'BYMA', 'mc': '=0 (0%)', 'pcv': 0, 'pcp': 0, 'industrySector': 'Government', 'industrySubgroup': 'Sovereign', 'industryGroup': 'Sovereign', 'currencies': [['S30S5', '1', 'ARS'], ['S2S5D', '2', 'USD'], ['S2S5C', '4', 'CCL']], 'f': '10:48'}, {'id': 'S2S5D-0001-C-CT-USD', 'deferredLevel': 0, 'SecurityID': 'S2S5D-0001-C-CT-USD', 'ticker': 'S2S5D', 'descripcion': 'LETRAS DEL TESORO CAP $ V30/09/25', 'mo': 'Dolar MEP', 'idMo': 2, 'idPlazo': 0, 'plaz

In [86]:
letras = response.json()["cotizaciones"]

# Get all tickers that start with the letter 'S'
lecap_tickers = [letra["ticker"] for letra in letras if letra["ticker"].startswith("S")]

In [87]:
def get_lecap_history_dataframe(lecap_tickers, access_token, cookie):
    all_rows = []

    for ticker in lecap_tickers:
        data = get_ticker_data(ticker)
        history = data.get("historico", [])
        cupon_data = get_cupon(ticker)
        cupon_info = cupon_data.get("bond", {}) if cupon_data else {}

        # --- Extract coupon and maturity as requested
        coupon = cupon_info.get("coupon", None)
        print(coupon)
        maturity = cupon_info.get("maturity", None)
        print(maturity)

        # Optionally fallback to Bravo/Cotizacion for 'Descripcion' if 'bond' missing
        if not maturity:
            cotiz_info = cupon_data.get("Cotizacion", {}) if cupon_data else {}
            from re import search
            desc = cotiz_info.get("Descripcion") or cotiz_info.get("descripcion") or ""
            # Try to extract maturity from description text e.g. "VTO. 29/05/2026"
            m = search(r'VTO[\. ]+(\d{2})[/-](\d{2})[/-](\d{4})', desc)
            if m:
                d, mth, y = m.groups()
                maturity = f"{y}-{mth}-{d}"

        for day in history:
            row = {
                "ticker": ticker,
                "date": day.get("fecha"),
                "closing_price": day.get("preciocierre"),
                "coupon": coupon,
                "maturity": maturity
            }
            all_rows.append(row)

    df = pd.DataFrame(all_rows)
    return df

In [88]:
df = get_lecap_history_dataframe(lecap_tickers, access_token, cookie)
print(df.head())

Status code: 200
None
None
Status code: 200
47.76%
2025-09-30
Status code: 200
47.76%
2025-09-30
Status code: 200
32.88%
2025-10-31
Status code: 200
32.88%
2025-10-31
Status code: 200
26.4%
2025-11-10
Status code: 200
None
None
Status code: 200
26.4%
2025-11-10
Status code: 200
27.12%
2025-11-28
Status code: 200
None
None
Status code: 200
None
2025-11-28
Status code: 200
43.2%
2026-01-16
Status code: 200
None
None
Status code: 200
None
2026-01-16
Status code: 200
47.4%
2026-02-27
Status code: 200
28.2%
2026-05-29
Status code: 200
None
None
Status code: 200
47.76%
2025-09-30
Status code: 200
47.76%
2025-09-30
Status code: 200
32.88%
2025-10-31
Status code: 200
None
None
Status code: 200
32.88%
2025-10-31
Status code: 200
26.4%
2025-11-10
Status code: 200
None
None
Status code: 200
26.4%
2025-11-10
Status code: 200
27.12%
2025-11-28
Status code: 200
None
None
Status code: 200
None
2025-11-28
Status code: 200
43.2%
2026-01-16
Status code: 200
None
None
Status code: 200
None
2026-01-16
Sta

In [95]:
df_valid = df[df['coupon'].notna() & df['maturity'].notna()].copy()

# Coupon as a rate
df_valid['coupon_rate'] = df_valid['coupon'].str.replace('%','').astype(float) / 100.0

df_valid['date'] = pd.to_datetime(df_valid['date'])
df_valid['maturity'] = pd.to_datetime(df_valid['maturity'])

df_valid['days_to_maturity'] = (df_valid['maturity'] - df_valid['date']).dt.days
df_valid = df_valid[df_valid['days_to_maturity'] > 0]

df_valid['TIR'] = ((1 + df_valid['coupon_rate']) / df_valid['closing_price']) ** (365 / df_valid['days_to_maturity']) - 1

df_valid['TIR_%'] = df_valid['TIR'] * 100
df_valid['TNA_%'] = df_valid['coupon_rate'] * 100

print(df_valid[['ticker','date','closing_price','coupon','maturity','TIR_%','TNA_%']])

     ticker       date  closing_price  coupon   maturity         TIR_%  TNA_%
191   S2S5D 2024-11-26       0.000001  47.76% 2025-09-30  2.047979e+09  47.76
192   S2S5D 2024-11-27       0.000001  47.76% 2025-09-30  2.163420e+09  47.76
193   S2S5D 2024-11-28       0.000001  47.76% 2025-09-30  2.286188e+09  47.76
194   S2S5D 2024-11-29       0.000001  47.76% 2025-09-30  2.416797e+09  47.76
195   S2S5D 2024-12-02       0.000001  47.76% 2025-09-30  2.861433e+09  47.76
...     ...        ...            ...     ...        ...           ...    ...
3025  S29Y6 2025-09-15       0.000654   28.2% 2026-05-29  4.944308e+06  28.20
3026  S29Y6 2025-09-16       0.000663   28.2% 2026-05-29  5.058452e+06  28.20
3027  S29Y6 2025-09-17       0.000650   28.2% 2026-05-29  5.431209e+06  28.20
3028  S29Y6 2025-09-18       0.000611   28.2% 2026-05-29  6.199843e+06  28.20
3029  S29Y6 2025-09-19       0.000594   28.2% 2026-05-29  6.747589e+06  28.20

[2392 rows x 7 columns]
